---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    ## Assume all dates in xx/xx/xx format are mm/dd/yyyy
    global df
    
    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    type1 = r'(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})'
    extractedDates = df.str.extractall(type1)
    not_duplicated = ~df.index.isin([index[0] for index in extractedDates.index])
    
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    type2 = r'(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2})'
    extractedDates = extractedDates.append(df[not_duplicated].str.extractall(type2), sort=True)
    not_duplicated = ~df.index.isin([index[0] for index in extractedDates.index])
    
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    type3 = r'(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4})'
    extractedDates = extractedDates.append(df[not_duplicated].str.extractall(type3), sort=True)
    not_duplicated = ~df.index.isin([index[0] for index in extractedDates.index])
    
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    type4 = r'(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4})'
    extractedDates = extractedDates.append(df[not_duplicated].str.extractall(type4), sort=True)
    not_duplicated = ~df.index.isin([index[0] for index in extractedDates.index])

    # Feb 2009; Sep 2009; Oct 2010
    type5 = '(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4})'
    without_day = df[not_duplicated].str.extractall(type5)
    # 6/2008; 12/2009
    type6 = r'(?P<month>\d\d?)/(?P<year>\d{4})'
    without_day = without_day.append(df[not_duplicated].str.extractall(type6), sort=True)
    without_day['day'] = 1
    extractedDates = extractedDates.append(without_day, sort=True)
    not_duplicated = ~df.index.isin([index[0] for index in extractedDates.index])
    
    # 2009; 2010
    type7 = r'(?P<year>\d{4})'
    without_month = df[not_duplicated].str.extractall(type7)
    without_month['day'] = 1
    without_month['month'] = 1
    extractedDates = extractedDates.append(without_month, sort=True)
    not_duplicated = ~df.index.isin([index[0] for index in extractedDates.index])
    

    # month
    extractedDates['month'] = extractedDates['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    monthsDictionary = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 
                        'nov':11, 'dec':12, 'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8,
                        'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12,
                        'january':1, 'february':2, 'march':3 , 'april':4, 'may':5, 'june':6, 'july':7,'august':8,
                        'september':9,'october':10, 'november':11, 'december':12, 'decemeber':12,
                        'January':1, 'February':2, 'March':3 , 'April':4, 'May':5, 'June':6, 'July':7,
                        'August':8, 'September':9,'October':10, 'November':11, 'December':12,
                        'Janaury':1, 'Since':1, 'Age': 8, 'Decemeber':12} ## typos as this is a raw, real-life derived dataset.
    extractedDates = extractedDates.replace(monthsDictionary)
    extractedDates['month'] = extractedDates['month'].apply(lambda month: str(month))
    
    # day
    extractedDates['day'] = extractedDates['day'].apply(lambda day: str(day))

    # year
    extractedDates['year'] = extractedDates['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    extractedDates['year'] = extractedDates['year'].apply(lambda day: str(day))
    
    # cleaned Dates, final result
    extractedDates['date'] = pd.to_datetime(extractedDates['month']+'/'+extractedDates['day']+'/'+extractedDates['year'])
    extractedDates.sort_values(by='date', inplace=True)
    
    # Your answer here
    return pd.Series(list(extractedDates.index.labels[0]))
#     return(extractedDates)
date_sorter()

C:\Users\Mahmoud\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.


0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64